In [ ]:
import yaml
import boto3
import h5py
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio

In [ ]:
# general structure for scenes in s3
# s3_bucket/software/dem/scene/scene_files
s3_bucket = 'deant-data-public-dev'
aws_credentials = 'credentials/credentials_aws.yaml'

# get crededentials for AWS
with open('aws_credentials.txt') as f:
    ACCESS_ID, ACCESS_KEY = f.readlines()
    ACCESS_ID = ACCESS_ID.strip()
    ACCESS_KEY = ACCESS_KEY.strip()

# setup s3
s3 = boto3.client(
    's3', 
    region_name='ap-southeast-2',
    aws_access_key_id=ACCESS_ID,
    aws_secret_access_key= ACCESS_KEY)

# make data directory to store local files
data_dir = 'data/scratch'
os.makedirs(data_dir, exist_ok=True)

In [ ]:
# set the scene we want to look at
software = 'rtc-opera'
dem = 'glo_30'
crs = '3031'
scene = 'S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B'
h5_file = f'OPERA_L2_RTC-{scene}.h5'.replace('no_rtc','')
#h5_file = f'OPERA_L2_RTC-{scene}_HH.tif'.replace('no_rtc','')
s3_prefix = f'{software}/{dem}/{crs}/{scene}/{h5_file}'
local_path = f'{data_dir}/{h5_file}'
# download file
download = True
if download:
    print(f'downloading {s3_prefix}')
    if not os.path.exists(local_path):
        s3.download_file(s3_bucket, s3_prefix, local_path)


In [ ]:
# load the h5 file
h5_data = h5py.File(local_path,'r')
print('keys')
print(list(h5_data))
print('data keys')
print(list(h5_data['data'].keys()))
print('metadata keys')
print(list(h5_data['metadata'].keys()))
print(list(h5_data['metadata']['processingInformation']['parameters'].keys()))

In [ ]:
def plot_array(
    arr, 
    cmap='binary_r',
    vmin=None,
    vmax=None,
    title='',
    ylabel='',
    save_path=''):

    stats_arr = np.array(arr)[np.array((np.isfinite(arr)))]
    print('Array Stats (excl nodata)')
    print(f'min: {stats_arr.min()}', 
        f'max: {stats_arr.max()}',
        f'mean: {stats_arr.mean()}',
        f'median: {np.percentile(stats_arr, 50)}',
        f'5th percentile: {np.percentile(stats_arr, 5)}',
        f'90th percentile: {np.percentile(stats_arr, 95)}',
        )

    # calculate percentiles if vmin or vmax is set as 'PXX'
    # e.g. vmin = 'P5'
    if vmin:
        if str(vmin)[0].upper() == 'P':
            vmin = np.percentile(stats_arr,int(vmin[1:]))
    if vmax:
        if str(vmax)[0].upper() == 'P':
            vmax = np.percentile(stats_arr,int(vmax[1:]))

    f, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(arr, 
            vmin = vmin, 
            vmax = vmax,
            cmap = cmap)
    plt.title(title)
    f.colorbar(im, ax=ax, label=ylabel)
    if save_path:
        plt.savefig(save_path)

In [ ]:
for i,k in enumerate(h5_data['data'].keys()):
    print(k)
    try:
        print(h5_data['data'][k].shape)
        plot_array(h5_data['data'][k],
                save_path=os.path.join(data_dir,f'{scene}_{k}.jpg'),
                vmin='P5',
                vmax='P95')
    except:
        f'skipping, no image for: {k}'
    # if i == 0:
    #     break

### Comparing Gamma, Beta and the rtcAreaNormalizationFactorGamma0ToBeta0 Layer

In [ ]:
with rasterio.open('data/scratch/OPERA_L2_RTC-S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B_HH.tif') as src:
    gamma = src.read(1)
with rasterio.open('data/scratch/no_rtcOPERA_L2_RTC-S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B_HH.tif') as src:
    beta = src.read(1)
rtcAreaNormalizationFactorGamma0ToBeta0 = np.array(h5py.File(
    'data/scratch/OPERA_L2_RTC-S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B.h5',
    'r')['data']['rtcAreaNormalizationFactorGamma0ToBeta0'])
dem = np.array(h5py.File(
    'data/scratch/OPERA_L2_RTC-S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B.h5',
    'r')['data']['interpolatedDem'])

#match nodata
gamma[np.isnan(beta)] = np.nan
beta[np.isnan(gamma)] = np.nan

print(gamma.shape)
print(beta.shape)
print(rtcAreaNormalizationFactorGamma0ToBeta0.shape)


Single pixel estimated value

In [ ]:
x, y = 5802, 4710 # get pixel from layers 
gamma_val = gamma[x,y]
beta_val = beta[x,y]
GammatoBeta_val = rtcAreaNormalizationFactorGamma0ToBeta0[x,y]
beta_est = gamma_val*GammatoBeta_val
print(f'pixel {x},{y}')
print(f'gamma: {gamma_val}')
print(f'beta: {beta_val}') 
print(f'rtcAreaNormalizationFactorGamma0ToBeta0: {GammatoBeta_val}')
print(f'beta_est = gamma * rtcAreaNormalizationFactorGamma0ToBeta0: {beta_est}')
print(f'beta - beta_est = {beta_est - beta_val}')
print(f'% err = {(100*(beta_est - beta_val)/beta_val)}')


In [ ]:
# calculate estimate for full array
beta_est = gamma*rtcAreaNormalizationFactorGamma0ToBeta0
# percentage error of estimate from gammatobeta later
beta_est_err = 100*((beta-beta_est)/beta) 

# gamma 
gamma_est = beta*(1/rtcAreaNormalizationFactorGamma0ToBeta0)
gamma_est_err = 100*((gamma-gamma_est)/gamma) 

In [ ]:
print('Percentage error of estimate beta from gamma')
plot_array(gamma_est_err,
                vmin=-20,
                vmax=20,
                cmap='bwr')
print('rtcAreaNormalizationFactorGamma0ToBeta0')
plot_array(rtcAreaNormalizationFactorGamma0ToBeta0,
                vmin='P5',
                vmax='P95',
                cmap='bwr')
print('dem')
plot_array(dem,
                vmin='P5',
                vmax='P99',
                cmap='binary')
plot_array(gamma,
                vmin='P5',
                vmax='P95',
                cmap='binary')


Two Layers needed from SLC
1. Lookup table - rdr2geo usig DEM
2. Radiometric Ratio Layer

1 layer needed for GSLC?
1. Radiometric Ratio Layer

Use coregistered CSLCS - these are geolocated
1. Radiometric Ratio Layer 

Is there an advantage of storing a ratio product opposed to the rtc layer itself?